In [3]:
# !pip install findspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [125]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [1]:
#Testing pyspark installation
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark-3.3.1-bin-hadoop3'

In [2]:
#Initiate Spark Context
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local[*]')
sc = pyspark.SparkContext(conf=conf)
spark  = SparkSession(sc)

In [3]:
spark

In [36]:
#Stop the Session
sc.stop()

In [4]:
from impala.dbapi import connect
conn = connect(host='192.168.118.137', port=21050)
cursor = conn.cursor()
cursor.execute('select * from vel_riskfactor')
# print(cursor)  # prints the result set's schema
results = cursor.fetchall()
for data in results:
        print(data)

('unsafe following distance', 54.0, 6.323071002960205)
('lane departure', 45.0, 3.116119146347046)
('overspeed', 86.0, 4.442246437072754)
('overspeed', 89.5, 7.700280666351318)
('overspeed', 85.0, 10.987820625305176)
('overspeed', 82.5, 6.337808609008789)
('unsafe tail distance', 27.0, 7.455998420715332)
('unsafe tail distance', 40.0, 9.151140213012695)
('unsafe following distance', 68.0, 2.92341685295105)
('unsafe tail distance', 63.0, 5.985864162445068)
('lane departure', 48.0, 1.5608904361724854)
('unsafe tail distance', 54.0, 7.513645172119141)
('unsafe following distance', 54.0, 4.575402736663818)
('overspeed', 90.0, 7.748658180236816)
('unsafe following distance', 48.0, 4.648604393005371)
('overspeed', 85.66666666666667, 7.673183917999268)
('lane departure', 24.0, 8.92822551727295)
('unsafe tail distance', 53.0, 3.064993381500244)
('unsafe tail distance', 48.66666666666666, 30.101442337036133)
('overspeed', 88.0, 4.7015604972839355)
('overspeed', 86.0, 6.118200302124023)
('oversp

In [5]:
df = spark.createDataFrame(results)

In [6]:
df.head()

Row(_1='driverid', _2='truckid', _3='model', _4=None, _5=None, _6=None, _7=None, _8=None, _9=None, _10=None, _11=None, _12=None, _13=None, _14=None, _15=None, _16=None, _17=None, _18=None, _19=None, _20=None, _21=None, _22=None, _23=None, _24=None, _25=None, _26=None, _27=None, _28=None, _29=None, _30=None, _31=None, _32=None, _33=None, _34=None, _35=None, _36=None, _37=None, _38=None, _39=None, _40=None, _41=None, _42=None, _43=None, _44=None, _45=None, _46=None, _47=None, _48=None, _49=None, _50=None, _51=None, _52=None, _53=None, _54=None, _55=None, _56=None, _57=None, _58=None, _59=None, _60=None, _61=None, _62=None, _63=None, _64=None, _65=None, _66=None, _67=None, _68=None, _69=None, _70=None, _71=None, _72=None, _73=None, _74=None, _75=None, _76=None, _77=None, _78=None, _79=None, _80=None, _81=None, _82=None, _83=None, _84=None, _85=None, _86=None, _87=None, _88=None, _89=None, _90=None, _91=None, _92=None, _93=None, _94=None, _95=None, _96=None, _97=None, _98=None, _99=None, _

In [5]:
from impala.util import as_pandas

cursor.execute('SELECT * FROM riskfactor')
riskfactor = as_pandas(cursor)

In [7]:
riskfactor.head()

driverid  events  totmiles  riskfactor
0       A1       3    628507    4.773216
1       A2       1    664543    1.504794
2       A3       6    639584    9.381098
3       A4       6    663289    9.045831
4       A5       8    676574   11.824280

In [9]:
riskfactor = riskfactor[['events','totmiles','riskfactor']]

In [10]:
riskfactor.to_csv('riskfactor.csv')

In [12]:
dataset = spark.read.csv("riskfactor.csv", inferSchema=True, header =True)
dataset.printSchema()

root
 |-- events: integer (nullable = true)
 |-- totmiles: integer (nullable = true)
 |-- riskfactor: double (nullable = true)



In [22]:
dataset.stat.corr('totmiles','riskfactor')

-0.10440426959034756

In [23]:
dataset.stat.corr('events','riskfactor')

0.9990495538557645

In [30]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['events'], outputCol = "Independent Features")
rf_df = vectorAssembler.transform(dataset)
rf_df = rf_df.select(['Independent Features', 'riskfactor'])
rf_df.show(5)

+--------------------+-----------+
|Independent Features| riskfactor|
+--------------------+-----------+
|               [3.0]|4.773216248|
|               [1.0]|1.504793525|
|               [6.0]|9.381097794|
|               [6.0]|9.045830727|
|               [8.0]|11.82427979|
+--------------------+-----------+
only showing top 5 rows



In [31]:
splits = rf_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [32]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'Independent Features', labelCol='riskfactor')
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [1.5462346059801306]
Intercept: -0.005527578414543197


In [33]:
lr_model.summary

In [34]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.156602
r2: 0.996950


In [35]:
pred_results = lr_model.evaluate(test_df)
pred_results.predictions.show(50)

+--------------------+-----------+------------------+
|Independent Features| riskfactor|        prediction|
+--------------------+-----------+------------------+
|               [1.0]|1.495617867|1.5407070275655874|
|               [1.0]|1.532761931|1.5407070275655874|
|               [1.0]|1.565861702|1.5407070275655874|
|               [2.0]|2.923416853| 3.086941633545718|
|               [2.0]|3.061980486| 3.086941633545718|
|               [2.0]|3.064993382| 3.086941633545718|
|               [2.0]|3.075328588| 3.086941633545718|
|               [2.0]|3.089700222| 3.086941633545718|
|               [2.0]|3.104857445| 3.086941633545718|
|               [2.0]|3.123599291| 3.086941633545718|
|               [3.0]|4.783025742|  4.63317623952585|
|               [3.0]|4.790396214|  4.63317623952585|
|               [3.0]|4.824780464|  4.63317623952585|
|               [4.0]|5.872723103|  6.17941084550598|
|               [4.0]|6.113225937|  6.17941084550598|
|               [4.0]|6.1345

In [14]:
import pyforest

In [18]:
from pyspark.ml.regression import LinearRegression


lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

lrModel = lr.fit(riskfactor[['events','totmiles']])

AttributeError: 'DataFrame' object has no attribute '_jdf'

In [6]:
geolocation.head()

truckid  driverid      event   latitude   longitude        city  \
0  truckid  driverid      event        NaN         NaN        city   
1      A54       A54     normal  38.440467 -122.714431  Santa Rosa   
2      A20       A20     normal  36.977173 -121.899402       Aptos   
3      A40       A40  overspeed  37.957702 -121.290780    Stockton   
4      A31       A31     normal  39.409608 -123.355566     Willits   

        state  velocity  event_ind  idling_ind  
0       state       NaN        NaN         NaN  
1  California      17.0        0.0         0.0  
2  California      27.0        0.0         0.0  
3  California      77.0        1.0         0.0  
4  California      22.0        0.0         0.0

In [8]:
drivermileage.head()

driverid  totmiles
0       A1    628507
1      A10    675377
2     A100    634338
3      A11    652452
4      A12    668241

In [9]:
geolocation.query("event != 'normal' & event_ind == 1 & idling_ind == 0")

truckid driverid                      event   latitude   longitude  \
3        A40      A40                  overspeed  37.957702 -121.290780   
13        A5       A5  unsafe following distance  41.526509 -124.038407   
15       A48      A48                  overspeed  38.752124 -121.288006   
25       A49      A49             lane departure  39.150171 -123.207783   
86       A97      A97                  overspeed  38.440467 -122.714431   
...      ...      ...                        ...        ...         ...   
7877     A27      A27  unsafe following distance  39.409608 -123.355566   
7895     A90      A90  unsafe following distance  39.017396 -122.057748   
7943     A84      A84                  overspeed  33.663357 -116.310009   
7977     A97      A97       unsafe tail distance  36.977173 -121.899402   
7978     A77      A77                  overspeed  34.136119 -117.865339   

            city       state  velocity  event_ind  idling_ind  
3       Stockton  California      77.0        1.0         0.0  
13       Klamath  California      33.0        1.0         0.0  
15     Roseville  California      77.0        1.0         0.0  
25         Ukiah  California      41.0        1.0         0.0  
86    Santa Rosa  California      33.0        1.0         0.0  
...          ...         ...       ...        ...         ...  
7877     Willits  California      54.0        1.0         0.0  
7895    Arbuckle  California      31.0        1.0         0.0  
7943   La Quinta  California      81.0        1.0         0.0  
7977       Aptos  California      66.0        1.0         0.0  
7978    Glendora  California      88.0        1.0         0.0  

[391 rows x 10 columns]